In [ ]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import model_from_json

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Concatenate
import keras.layers as lyr
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint
from keras.layers import Bidirectional
import tensorflow as tf



/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


# New Section

In [ ]:
TRAIN_CSV = '/questions.csv'
#TRAIN_CSV = 'sample.csv'
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'

In [ ]:
import nltk
nltk.download('stopwords')
train_df = pd.read_csv(TRAIN_CSV)
stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"


--2020-07-24 07:17:55--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.81.22
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.81.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  16.2MB/s    in 1m 50s  

2020-07-24 07:19:46 (14.3 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
def stringToWordList(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\(", " ( ", text)
    text = re.sub(r"\)", " ) ", text)
    text = re.sub(r"\?", " ? ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = text.split()
    return text


In [ ]:
questions_cols = ['question1', 'question2']
vocab = dict()
maxSeqLength = 0
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

for index, row in train_df.iterrows():
    for question in questions_cols:
        q2num = []
        for word in stringToWordList(row[question]):
            if word in stops and word not in word2vec.vocab:
                continue
            if word not in vocab:
                vocab[word] = len(vocab)+1
            q2num.append(vocab[word])
        train_df.at[index,question] = q2num
        if len(q2num) > maxSeqLength:
            maxSeqLength = len(q2num)

embeddingDim = 300
embeddings = 1 * np.random.randn(len(vocab) + 1, embeddingDim)
embeddings[0] = 0

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# Build the embedding matrix
for word, index in vocab.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

del word2vec

In [ ]:
test_size = 20000
validation_size = 40000
training_size = len(train_df) - validation_size

X = train_df[questions_cols]
Y = train_df['is_duplicate']

X_train_val, X_test, Y_train_val, Y_test = train_test_split(X, Y, test_size=test_size)
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train_val, Y_train_val, test_size=validation_size)

# Split to dicts
X_train = {'left': X_train.question1, 'right': X_train.question2}
X_test = {'left': X_test.question1, 'right': X_test.question2}
X_validation = {'left': X_validation.question1, 'right': X_validation.question2}

# Convert labels to their numpy representations
Y_train = Y_train.values
Y_test = Y_test.values
Y_validation = Y_validation.values

# Zero padding
for dataset, side in itertools.product([X_train, X_validation, X_test], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=maxSeqLength)

# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)


In [ ]:
from keras.layers import dot, Dense , Flatten , Reshape , add , Dropout
from keras.layers.normalization import BatchNormalization

n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 1024
n_epoch = 2

DROPOUT =  0.1

question1 = Input(shape=(maxSeqLength,))
question2 = Input(shape=(maxSeqLength,))

q1 = Embedding(len(embeddings), 
                 embeddingDim,
                 weights=[embeddings], 
                 input_length=maxSeqLength, 
                 trainable=False)(question1)
q1 = Bidirectional(LSTM(n_hidden, return_sequences=True), merge_mode="sum")(q1)

q2 = Embedding(len(embeddings), 
                 embeddingDim, 
                 weights=[embeddings], 
                 input_length=maxSeqLength, 
                 trainable=False)(question2)
q2 = Bidirectional(LSTM(n_hidden, return_sequences=True), merge_mode="sum")(q2)

attention = dot([q1,q2], [1,1])
attention = Flatten()(attention)
attention = Dense((maxSeqLength*n_hidden))(attention)
attention = Reshape((maxSeqLength, n_hidden))(attention)

merged = add([q1,attention])
merged = Flatten()(merged)
merged = Dense(200, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)
merged = Dense(200, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)
merged = Dense(200, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)
merged = Dense(200, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)

is_duplicate = Dense(1, activation='sigmoid')(merged)

model = Model(inputs=[question1,question2], outputs=is_duplicate)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Start training
training_start_time = time()

lstm_trained = model.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size,epochs=n_epoch,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))



Train on 344351 samples, validate on 40000 samples
Epoch 1/2
344351/344351 [==============================] - 5264s 15ms/step - loss: 0.5288 - accuracy: 0.7306 - val_loss: 0.4691 - val_accuracy: 0.7664
Epoch 2/2
344351/344351 [==============================] - 4986s 14ms/step - loss: 0.4402 - accuracy: 0.7863 - val_loss: 0.4476 - val_accuracy: 0.7839
Training time finished.
2 epochs in 2:50:55.662115


In [ ]:
scores = model.evaluate([X_test['left'], X_test['right']], Y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

accuracy: 78.62%
Saved model to disk
